**Anmerkung**

Dieses Notebook hat zum Ziel die Daten vor einer Anpassungen, also ohne entfernen von Daten die ausserhalb des Projektzeitraumes sind oder einen Fehler beinhalten, zu betrachten und kennenzulernen. Da die meisten Data Wrangling und Plot-Funktionen auch im Analyse-Bericht für die ETH verwendet werden, sind diese Funktionen auf die Files `data_wrangling.py` und `plot_functions.py` ausgelagert. Einige Grafiken im Notebook `analysis.ipynb` sind ebenfalls aus einer Form von EDA entstanden. 

# Explorative Datenanalyse der Sensordaten aus dem Projekt "vacancy no-vacancy"

**Autor**: Roman Studer: roman.studer1@students.fhnw.ch, Simon Luder: simon.luder@students.fhnw.ch

**Projekbeschreibung**: https://ds-spaces.technik.fhnw.ch/fs20p4/

Diese Arbeit befasst sich mit der Auswertung von Sensordaten welche im Rahmen des Projektes "vacancy no-vacancy" durch die ETH Zürich gesammelt wurden.
Für die Dauer eines Jahres steht seit dem Sommer 2019 das Mock-Up einer Wohnung auf der Dachterrasse des HIL-Gebäudes der ETH Zürich. Personen unterschiedlicher sozialer Gruppen wohnen für je eine Woche im Mock-Up.

Im Mock-up sind vier beweglichen Elemente (Drehwand, Lampe Drehwand, Drehschrank, Lampe Aussenwand) prototypisch eingesetzt. Sensoren messen die Bewegungshäufigkeit, die Zeitpunkte sowie die jeweils gewählten Winkel sämtlicher beweglicher Bauteile. Türen und Schubladen sind ebenfalls mit Sensoren ausgestattet, welche eine Interaktion mit den Elementen binär festhält (ON/OFF oder OPEN/CLOSED). Die Auswertung der erhobenen Daten gibt Aufschlüsse über das dynamische Zusammenspiel zwischen Menschen und Wohnung. Mehr zum Projekt finden Sie auf der [Website](https://arch.ethz.ch/news-und-veranstaltungen/lehre-forschung/vacancy-novacancy.html) der ETH Zürich.

Der Fokus dieser Datenanalyse liegt auf den Drehelementen. Diese werden durch den Raum beschränkt. Die Ausgangslage ist bei jedem Drehelement 0°, wobei eine Drehung im Uhrzeigersinn erfolgt. Somit können die Bewohner den Raum individuell nach Ihren Bedürfnissen gestalten.

Die Datenanalyse beginnt mit der Überprüfung der Drehsensorwerte, um diese allenfalls zu korrigieren oder zu ignorieren.

![Lageplan Mock-up](layout.PNG)

In [ ]:
# IMPORTS:
import os.path

# data wrangling
from data_wrangling import *
import pandas as pd
import numpy as np
import datetime, inspect
from os import path

# plotting
from plot_functions import * # own plot functions from plot_functions.py
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

color = [clrs['Drehwand'][0], clrs['Drehschrank'][0], clrs['LampeAussenwand'][0], clrs['LampeDrehwand'][0]]

## 1. Datenimport

Import von log_sensor table als `df_log`, person table als `df_person` und occupation_period als `df_op`. Zusätzlicher import von CSVs welche Informationen über Zeiträume enthält während denen die Daten entweder Fehlerhaft sind (`mal`) oder die Daten ausserhalb des ofiziellen Zeitraumes sind da zu diesem Zeitpunkt kein Partizipant im mock_up sind.

In [ ]:
# data import
dba = DBAccess() # connect to db and load new data

df_log = dba.select('sensor_data_total') # unfiltered log_sensor data
df_person = dba.select('person') # dataframe containing information to every participant
df_op = dba.select('occupation_period')
person = dba.select('person') # dataframe containing information to every participant


# import csv's
mal = pd.read_csv('../database/Malfunction.csv') # table with malfunction timeframes
not_o = pd.read_csv('../database/not_occupied.csv') # table with information about missing timeframes

# enrich person with cingle and couple information
df_person = living_situation_person(df_person)

In [ ]:
df_log.head()

In [ ]:
df_log.tail()

In [ ]:
df_person.sample()

In [ ]:
df_op.sample()

## 3. Datenreinigung

#### Datentypen
Prüfen der Datentypen des DataFrame `df_log` sowie Anpassung für die weitere Analyse.

In [ ]:
df_log.info() # object --> 'string'

#### Duplizierte Zeilen
Entfernen doppelter Einträge

In [ ]:
dub = len(df_log[df_log.duplicated()])
print(f'Anzahl duplizierte Zeilen: {dub}')

if dub > 0:
    df_log.drop_duplicates()

#### Fehlende Datenpunkte
Features im DataFrame `df_log` können NaN Werte enthalten. 
- Sensoren die nicht vom Typ 'turn' (Drehsensoren) sind haben keine Einträge für "sensor_numeric1", "sensor_numeric2", "sensor_diff", "sensor_duration"
- Sensoren vom Typ 'turn' haben keine Einträge für 'sensor_state'

In [ ]:
df_log.isna().sum()

Datenpunkte mit fehlenden Informationen über occupation_period_id, log_sensor_id und sensor werden nicht weiter verwendet.

In [ ]:
df_log.dropna(subset=['occupation_period_id','sensor','log_sensor_id'], inplace=True)

In [ ]:
df_log.isna().sum()

# Drehsensoren
Insgesamt vier Sensoren sind vom sensor_type 'turn', sprich Drehsensoren welche die vier in der oberen Grafik dargestellten Sensoren darstellen. "Drehwand","Drehschrank", "LampeDrehwand", "LampeAussenwand". Der Fokus der Arbeit liegt auf diesen Sensoren. 

## Postition der Drehsensoren durch die gesamte Projektzeit, unbehandelt

In [ ]:
p_rotary(sensor_states)

**Lesebeispiel:** Die Darstellung zeigt interaktiv die Stellung aller Drehsensoren über die gesamte Projektdauer an. Die x-Achse gibt Datum, oder beim Heranzoomen auch Urzeit, an. y-Achse gibt den jeweiligen Winkel des Sensors in Grad an. Mit der Maus kann ein Teilstück der Grafik angewählt werden (Click & Drag) sowie der Sensor angezeigt werden (Hover). Um wider zurück auf die Gesamtansicht zu kommen, oben links auf den Knopf "Alle" klicken. In der oberen rechten Ecke der Grafik kann mittels eines Dropdown alle oder einzelne Sensoren ausgewählt werden. Bei einzelner Ansicht stellen die zwei orangen Linien die Wertegrenze des jeweiligen Sensors dar.

**Feststellungen:**
- Anhand der oberen Grafik ist gut zu erkennen das zwischen November und Februar Doe Drehwand sich 1. Ausserhalb der Minimal- und Maximalwerte befindet, 2. Im November konstant zu hoch ist, 3. Anfangs Dezember konstant zu tief und eine verkleinerte Range der Werte von nur ca. 60 Grad, 4. Anfangs November und Anfangs Februar extrem ausschlägt.
- Ab Mitte Märt bis Anfangs Mai ist die Unterbrechung des Projektes durch den Ausbruch des Covid-19 Virus zu erkennen. 
- Alle Sensoren weisen immer wieder schiefe Linien dar. Der folgende Anfangswert ist also nicht auf der gleichen Position wie der letzte Endwert.
- Immerwieder sind extrem kleine Sprünge von weniger als ein paar Grad zu erkennen.

## Postition der Drehsensoren durch die gesamte Projektzeit, korrigiert
Hier handelt es sich um einen Vorgriff aus dem `analyse.ipynb`-Notebook um darzustellen was aus den oberen Feststellungen entstanden ist, im weiteren Verlauf des EDA-Notebooks werden allerdings noch die unkorrigierten Daten verwendet, Der folgende Lineplot enthält die selben Daten wie die obere Grafik, mit der Ausnahme das fehlerhafte Werte entfernt wurden. Dazu wurden die Daten geflagged.

Diese Darstellung hat den Zweck, die Auswirkungen der oben genannten Behandlungsmöglichkeiten aufzuzeigen. Hierbei wurden folgende Entscheide getroffen:
- Es werden lediglich die Endwerte berücksichtigt
- Daten, welche für die Positionsanalyse ignoriert werden, jedoch nicht für die Häufigkeitsanalyse, werden hier nicht aufgezeigt (obwohl dies Teil der Analyse sind)
- Extremwerte ausserhalb des Wertebereiches eines Drehsensors werden auf das Minimum/Maximum +/- die definierte Toleranz zurückgestuft.

Somit ist die Auswirkung der ignorierten und korrigierten Zeitintervalle sowie der korrigierten Extremwerte ersichtlich. Weitere Zeitintervalle, welche für die Analyse ignoriert werden müssen, wie Spezialanlässe oder ein unbesetztes Mock-Up, sind hier nicht ersichtlich, da dieser Plot lediglich die Auswirkung der entschiedenen Behandlungsmöglichkeiten aufzeigen soll.

In [ ]:
p_rotary(prep_rotary_sensors)

## Positionen der Drehsensoren

In [ ]:
fig = px.box(data_frame=df_log[df_log.sensor_type =='turn'],
            y='sensor_numeric1',
            color='sensor',
            title='Boxplot, Position der Drehsensoren',
            labels={'sensor_numeric1':'Winkel (Grad)'},
            color_discrete_sequence=color,
            orientation='v')

fig.show()

Auf der Grafik zu erkennen ist folgendes: (Stand Juni 2020)
- Drehwand und LampeDrehwand haben Werte die Stark under 0 Grad (also unter dem Minimalwert) sind.
- Die Drehwand (welche nicht 360° Spielraum hat) hat Werte über 360° und auch eine Range an Werten über 360°
- Drehschrank und LampeAussenwand sind korrekt, bzw. treten nicht über Max und Min Werte.
- Der Median aller vier Sensoren ist unter 80°. Eine Tendenz nach 0 also Ausgangspostion ist so evtl. möglich.

## Verteilung der Bewegungen

In [ ]:
fig = px.histogram(data_frame=df_log[df_log.sensor_type =='turn'],
            x='sensor_numeric1',
            color='sensor',
            title='Boxplot, Position der Drehsensoren',
            labels={'sensor_numeric1':'Winkel (Grad)'},
            color_discrete_sequence=color,
            orientation='v').update_layout(yaxis_title='Anzahl Bewegungen')

fig.show()

**Lesebeispiel:** Das obere Histogramm stellt die Häufigkeit einer Positionierung aller vier Drehsensoren dar. Anhand eines doppelklickes auf der Legende kann ein einzelnes Histogramm betrachtet werden. 

**Feststellungen:** (Stand Juni 2020)
- Drehwand:
    - Anhäufung bei 15-30°, 70-85°, 110-120°
    - Werte unter 0°
- Drehschrank:
    - Deutlich mehr werte auf dem Minimalwert, kann auf eine Unbeliebtheit hinweisen
- LampeDrehwand:
    - Anhäufung bei 25-30° und 40-45°
- Lampe Aussenwand:
    - Hohe Anzahl bei 0° und 80-85°

Die Auswertung aus dem Histogramm sollen kritisch betrachtet werden. Beliebte Positionen und Einstellungen sind auch abhängig von der Zeitdauer auf einer bestimmten Position. Ein gewichtetes Histogramm ist hier zu empfehlen.

## Häufigkeit der Bewegungen

In [ ]:
df_count = df_log[df_log.sensor_type =='turn'].groupby('sensor').count().reset_index()
fig = px.bar(data_frame=df_count,
            x='sensor',
            y='log_sensor_id',
            color='sensor',
            title='Boxplot, Position der Drehsensoren',
            labels={'sensor_numeric1':'Winkel (Grad)'},
            color_discrete_sequence=color).update_layout(yaxis_title='Anzahl Bewegungen')

fig.show()

(Stand Juni 2020) Anhand eines einfachen Barplots ist zu erkennen das rein von der Anzahl Bewegungen die Drehwand am meisten benutzt wird.

# Schubladen und Türen
Die meisten Sensoren sind keine Drehsensoren, sondern nehmen einen binären Wert an. Schulbaden, Schränke und Türen nehmen Werte von Open/Close oder On/Off an.
Für eine Bessere unterteilung werden die Sensoren in drei Gruppen unterteilt:
- Schubladen und Schränke
- Türen
- Küche

In [ ]:
# Kitchen sensors
df_k = df_log[df_log['room'] == 'K'] # K for 'Küche'

# cupboards and drawers
df_s = df_log[df_log['sensor'].isin(['S_Boden_Wand_cyr',
                             'S_Boden_Kueche_cyr',
                             'S_Schub_Wand_cyr ',
                             'S_Schub_Kueche_cyr',
                             'H_Putz_cyr',
                             'H_Graderobe_cyr',
                             'B_Schrank_cyr',
                             'B_Wasch_cyr',
                             'W_Schub_Bad_cyr',
                             'W_Schub_Wand_cyr',
                             'W_Boden_Bad_cyr',
                             'W_Boden_Wand_cyr'
                            ])]

# doors
df_d = df_log[df_log['sensor'].isin(['H_Tuer_Str',
                             'B_Tuer_Str'
                            ])]

## Küche

In [ ]:
df_count = df_k.groupby(by=['sensor', 'sensor_state']).count().reset_index()
fig = px.bar(data_frame=df_count,
            x='sensor',
            y='log_sensor_id',
            color='sensor',
            facet_row = 'sensor_state',
            title='Boxplot, Position der Drehsensoren',
            labels={'sensor_numeric1':'Winkel (Grad)'}).update_layout(yaxis_title='Anzahl Bewegungen')

fig.show()

Die Sensoren K_Abfall_cyr (Abfalleimer), K_Kuehl_cyr (Kühlschrank) und K_Schub_Oben_cyr stechen deutlich hervor. Jeder Sensor ist unterteilt in die jeweiligen States die er annemhen kann. Die Sensoren scheinen zu funktionieren.

## Schubladen und Schränke

In [ ]:
df_count = df_s.groupby(by=['sensor','sensor_state']).count().reset_index()
fig = px.bar(data_frame=df_count,
            x='sensor',
            y='log_sensor_id',
            facet_row = 'sensor_state',
            color='sensor',
            title='Boxplot, Position der Drehsensoren',
            labels={'sensor_numeric1':'Winkel (Grad)'}).update_layout(yaxis_title='Anzahl Bewegungen')

fig.show()

Die Garderobe und der Putzschrank stechen hier deutlich hervor. Die Sensoren sind ebenfalls in die States aufgeteilt die sie annehmen können.

## Türen 

In [ ]:
df_count = df_d.groupby('sensor').count().reset_index()
fig = px.bar(data_frame=df_count,
            x='sensor',
            y='log_sensor_id',
            color='sensor',
            title='Boxplot, Position der Drehsensoren',
            labels={'sensor_numeric1':'Winkel (Grad)'}).update_layout(yaxis_title='Anzahl Bewegungen')

fig.show()

Die Tür zum Balkon wird häufiger verwendet als die Eingangstür.

# Personen
Für eine spätere Analyse ist die Verteilung der Benutzergruppen, also Alter, Beruf oder Geschlecht interessant. Dies erlaubt es die Daten zu normieren um eine Interpretation zu erlauben.

In [ ]:
single_couple(df_person)

Das Verhältnis zwischen Singles und Paaren ist etwa gleich.

In [ ]:
sex_dist(df_person)

Insgesamt sind mehr Männer Teilnemher (9 mehr als Frauen)

In [ ]:
age_dist(person)

Je höher dass Alter umso weniger Teilnehmer.

In [ ]:
sex_age_dist(df_person)

Frauen in der Altersgruppe 30-45 sind deutlich unterrepräsentiert.

In [ ]:
couples_sex_age(df_person)

Bei Paaren sind keine gleichgeschlechtlige Paare in der Alterskategorie 45-60, 60-75 vorhanden.

In [ ]:
singles_sex_age_dist(df_person)

Es gibt keine männlichen Singles in der Altersgruppe 18-30 und 45-60. Die Männer dominieren allerdings die Anzahl Teilnehmer in der Gruppe 30-45.

In [ ]:
ss_couples_sex_age_dist(df_person)

Bei den Paaren fehlen Gleichgeschlechtliche Paare in den oberen drei Altergruppen

In [ ]:
profession_dist(person)

Viele Teilnehmer sind entweder Student (meist Architekturstudent) oder Architekt. 

# Schlussbemerkung
Dieses kurze EDA Notebook hat gute Einblicke in die Datenbeschaffenheit ermöglicht. Vor allem die Visualisierung der Positionen der Log-Sensoren war massgebend für den Umgang mit fehlerhaften Daten. Problematisch ist das es Benutzergruppen gibt die nicht repräsentiert sind. Diese Erkenntnis wurde an die ETH weitergeleitet. 